In [ ]:
import uproot
import glob
import numpy as np
import pandas as pd
from scipy.constants import epsilon_0, e as q_e
from scipy.interpolate import griddata


from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
from matplotlib.colorbar import ColorbarBase
from matplotlib.colors import Normalize
import matplotlib.colors as mcolors
from matplotlib.collections import LineCollection
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import trimesh
from trimesh.points import PointCloud

from common_functions import *

In [ ]:
stacked_spheres = trimesh.load_mesh('../sphere-charging/geometry/stacked_spheres_frompython.stl') 

# get a single cross section of the mesh
section = stacked_spheres.section(plane_origin=stacked_spheres.centroid, plane_normal=[0, 1, 0])
slice_2D, to_3D = section.to_2D()
slice_2D.show()

## Case 1: SW electrons and ions

In [ ]:
directory_path = "../build-sphere-charging/root/"
configIN = "solarwind"
filelist = sorted(glob.glob(f"{directory_path}/*stackediteration*{configIN}*num2000.root"))

all_incident_protons_inside, all_incident_electrons_inside = [],[]

for fileIN in filelist:

    print(fileIN.split("/")[-1])
    number_str = fileIN.split("/")[-1].split("_")[1]
    iterationNUM = int(''.join(filter(str.isdigit, number_str)))

    if iterationNUM > 28:
        break

    # read data from different iterations
    vars()["protons_inside_"+str(number_str)], vars()["electrons_inside_"+str(number_str)] = calculate_stats(read_rootfile(fileIN.split("/")[-1], directory_path=directory_path), 
                                                                                                             config=configIN)
    all_incident_protons_inside.append(vars()["protons_inside_"+str(number_str)])
    all_incident_electrons_inside.append(vars()["electrons_inside_"+str(number_str)])
    print(78*"-")

# Concatenate all iterations into single DataFrames
all_incident_protons_inside_df = pd.concat(all_incident_protons_inside, ignore_index=True)
all_incident_electrons_inside_df = pd.concat(all_incident_electrons_inside, ignore_index=True)

In [ ]:
# Concatenate all iterations into single DataFrames
all_incident_protons_inside_df = pd.concat(all_incident_protons_inside, ignore_index=True)
all_incident_electrons_inside_df = pd.concat(all_incident_electrons_inside, ignore_index=True)

In [ ]:
iterationIN=0
surface,facolors = plot_surface_potential_fornegativepositive_charge(all_incident_electrons_inside_df, all_incident_protons_inside_df, stacked_spheres, vmin=-0.2,vmax=0.2)

plt.hist(facolors[facolors!=0],bins=50)
plt.show()

# Make sure each triangle has its own unique vertices
surface_edited = surface.copy()
surface_edited.unmerge_vertices()
surface_edited.visual.vertex_colors = None
surface_edited.show()

In [ ]:
# Define colormap and normalization
cmap = plt.cm.seismic
norm = Normalize(vmin=-0.2, vmax=0.2)

# Create a figure and a single axis for the colorbar
fig, ax = plt.subplots(figsize=(6, 1))
fig.subplots_adjust(bottom=0.5)

# Create the colorbar
cb = ColorbarBase(ax, cmap=cmap, norm=norm, orientation='horizontal')
cb.set_label('Surface Potential (mV)')  # Optional label

plt.show()


In [ ]:
bbox_min = np.array([-0.25, -0.31, 0])
bbox_max = np.array([ 0.15,  0.1,  100])

# Filter vertices
in_box = np.all((surface_edited.vertices >= bbox_min) & 
                (surface_edited.vertices <= bbox_max), axis=1)

# Get face indices where all 3 vertices are in the box
face_mask = np.all(in_box[surface_edited.faces], axis=1)

# Extract submesh
cropped = surface_edited.submesh([face_mask], only_watertight=False, append=True)
cropped.show()

## Case 2: Photoemission (incident gammas)

In [ ]:
configIN = "photoemission"
directory_path = "../build-sphere-charging/root/"
filelist = sorted(glob.glob(f"{directory_path}/*stackediteration*{configIN}*num5000.root"))

all_gamma_holes = []
all_electrons_inside = []

for fileIN in filelist:
    print(fileIN.split("/")[-1])
    number_str = fileIN.split("/")[-1].split("_")[1]
    iterationNUM = int(''.join(filter(str.isdigit, number_str)))

    if iterationNUM > 33:
        break

    # read data from different iterations
    vars()["gamma_holes_"+str(number_str)], _, vars()["electrons_inside_"+str(number_str)] = calculate_stats(read_rootfile(fileIN.split("/")[-1], directory_path=directory_path),
                                                                                                             config=configIN)
    all_gamma_holes.append(vars()["gamma_holes_"+str(number_str)])
    all_electrons_inside.append(vars()["electrons_inside_"+str(number_str)])
    print(78*"-")

# Concatenate all iterations into single DataFrames
all_gamma_holes_df = pd.concat(all_gamma_holes, ignore_index=True)
all_electrons_inside_df = pd.concat(all_electrons_inside, ignore_index=True)

In [ ]:
# Load the data, skipping header if necessary
data = np.loadtxt('../build-sphere-charging/FieldMap.txt', skiprows=1)

# Extract columns
x, y, z = data[:, 0], data[:, 1], data[:, 2]
Ex, Ey, Ez = data[:, 3], data[:, 4], data[:, 5]

field = pd.DataFrame({"x":x,"y":y,"z":z,"Ex":Ex, "Ey":Ey, "Ez":Ez})
field["E_mag"]=np.linalg.norm(field[["Ex", "Ey", "Ez"]].values, axis=1)

In [ ]:
# Usage with black semi-transparent edges
scene = plot_trimesh_edges_only(stacked_spheres, edge_color=[0, 0, 0, 350])  # Black, 50% transparent

# Create a PointCloud object
gamma_photoemission_sites = trimesh.points.PointCloud(np.array(all_gamma_holes_df["Pre_Step_Position_mm"].tolist()), colors=[0, 255, 0, 255])  # RGBA green points
photoelectron_stopping_sites = trimesh.points.PointCloud(np.array(all_electrons_inside_df["Pre_Step_Position_mm"].tolist()), colors=[0, 0, 255, 255])  # RGBA blue points

scene.add_geometry([gamma_photoemission_sites])

max_arrows = 3000
field_nonzero = field[field["E_mag"] > 2e-7]
field_plot = field_nonzero #.sample(n=max_arrows, random_state=42) if len(field) > max_arrows else field

directions = field_plot[['Ex', 'Ey', 'Ez']].values
magnitudes = np.linalg.norm(directions, axis=1)
directions_unit = np.where(magnitudes[:, None] > 0, directions / magnitudes[:, None], 0)

# Normalize magnitudes to [0, 1] for colormap
norm_magnitudes = (magnitudes - magnitudes.min()) / (np.ptp(magnitudes) + 1e-12)

# Create a colormap
cmap = plt.cm.jet
colors_rgba = cmap(norm_magnitudes)  # shape: (N, 4) with floats in [0,1]

# Convert to 0–255 uint8 for trimesh
colors_rgba = (colors_rgba * 255).astype(np.uint8)

# Define base dimensions
base_arrow_length = 0.1  # This will be scaled by magnitude
arrow_radius = 0.001
cone_ratio = 0.2  # Cone length as fraction of total arrow length

# Normalize magnitudes for scaling arrow lengths
scaling_factor = 1.0  # Adjust this to control overall arrow size
scaled_lengths = base_arrow_length * scaling_factor * norm_magnitudes

for pos, dir_vec, color, magnitude_norm, scaled_length in zip(
    field_plot[['x', 'y', 'z']].values, 
    directions_unit, 
    colors_rgba, 
    norm_magnitudes,
    scaled_lengths
):
    if np.linalg.norm(dir_vec) == 0:
        continue

    # Calculate arrow dimensions based on scaled length
    arrow_length = scaled_length
    cone_length = arrow_length * cone_ratio
    shaft_length = arrow_length - cone_length

    # Skip very small arrows to avoid visualization issues
    # if arrow_length < 1e-4:
    #     continue

    # Create shaft - centered at origin along Z-axis
    shaft = trimesh.creation.cylinder(radius=arrow_radius, height=shaft_length, sections=12)
    shaft.apply_translation([0, 0, shaft_length / 2])  # Move so base is at z=0, top at z=shaft_length

    # Create cone - positioned to touch the shaft
    cone = trimesh.creation.cone(radius=arrow_radius * 2, height=cone_length, sections=12)
    # Position cone so its base touches the top of the shaft (at z=shaft_length)
    cone.apply_translation([0, 0, shaft_length])

    # Combine parts
    arrow = trimesh.util.concatenate([shaft, cone])

    # Set color per face
    arrow.visual.face_colors = np.tile(color, (arrow.faces.shape[0], 1))

    # Align +Z to direction
    transform = trimesh.geometry.align_vectors([0, 0, 1], dir_vec)
    transform[:3, 3] = pos
    arrow.apply_transform(transform)

    scene.add_geometry(arrow)

scene.show()

In [ ]:
field_nonzero = field[field["E_mag"] > 2e-7]

In [ ]:
len(field_nonzero)

In [ ]:
# Target point
target = np.array([0, -0.1, 0.20700000000000002])

# Compute Euclidean distance to all points in the field
distances = np.linalg.norm(field[['x', 'y', 'z']].values - target, axis=1)

# Find index of the closest point
closest_idx = np.argmin(distances)

# Extract the closest point and its field
closest_point = field.iloc[closest_idx]

print("Closest point:")
print(closest_point[['x', 'y', 'z']])
print("\nElectric field at that point:")
print(closest_point[['Ex', 'Ey', 'Ez']])


After iteration 1 (total e-: 1139228, Holes:     870236)

------------------------------------------
After iteration 2 (total e-: 8,660)

Closest point:
x    0.000000
y   -0.100000
z    0.206795
Name: 38481, dtype: float64

Electric field at that point:
Ex    1.360211e-07
Ey    1.542312e-08
Ez    8.020124e-08
Name: 38481, dtype: float64

------------------------------------------
After iteration 1 (total e-: 4,356)

Closest point:
x    0.000000
y   -0.100000
z    0.206795
Name: 4605855, dtype: float64

Electric field at that point:
Ex    8.230766e-08
Ey   -5.993845e-09
Ez    4.336823e-08
Name: 4605855, dtype: float64

In [ ]:
# define the size of the world
WorldX, WorldY, WorldZ = 200, 600, 606.61

# think we need to get to ~8 iterations at 1 million particles each to get to 3s 
((4*1e-6*(WorldX*WorldY)/(1e6**2))*6.241509*10**18*3)/1e6

In [ ]:
print("up through iteration 33")
surface,pot = plot_surface_potential_fornegativepositive_charge(all_electrons_inside_df, all_gamma_holes_df, stacked_spheres, vmin=-0.2,vmax=0.2)
print(min(pot),max(pot))
# Make sure each triangle has its own unique vertices
surface_edited = surface.copy()
surface_edited.unmerge_vertices()
surface_edited.visual.vertex_colors = None
surface_edited.show()

In [ ]:
bbox_min = np.array([-0.25, -0.31, 0])
bbox_max = np.array([ 0.15,  0.1,  100])

# Filter vertices
in_box = np.all((surface_edited.vertices >= bbox_min) & 
                (surface_edited.vertices <= bbox_max), axis=1)

# Get face indices where all 3 vertices are in the box
face_mask = np.all(in_box[surface_edited.faces], axis=1)

# Extract submesh
cropped = surface_edited.submesh([face_mask], only_watertight=False, append=True)
cropped.show()

In [ ]:
# Define colormap and normalization
cmap = plt.cm.seismic
norm = Normalize(vmin=-0.2, vmax=0.2)

# Create a figure and a single axis for the colorbar
fig, ax = plt.subplots(figsize=(4, 0.5))
fig.subplots_adjust(bottom=0.5)

# Create the colorbar
cb = ColorbarBase(ax, cmap=cmap, norm=norm, orientation='horizontal')
cb.set_label('Surface Potential (mV)')  # Optional label

plt.show()


## Case 3: all particles (incident e-, protons, gammas)

In [ ]:
directory_path = "../build-sphere-charging/root/"
configIN = "allparticles"
filelist = sorted(glob.glob(f"{directory_path}/*stackediteration*{configIN}*num5000.root"))

all_gamma_holes,all_photoemission_electrons,all_protons_inside,all_electrons_inside = [],[],[],[]

for fileIN in filelist:
    print(fileIN.split("/")[-1])
    number_str = fileIN.split("/")[-1].split("_")[1]
    iterationNUM = int(''.join(filter(str.isdigit, number_str)))

    # read data from different iterations
    vars()["gamma_holes_"+str(number_str)], vars()["photoemission_electrons_inside_"+str(number_str)], \
        vars()["protons_inside_"+str(number_str)], vars()["electrons_inside_"+str(number_str)] = calculate_stats(read_rootfile(fileIN.split("/")[-1], directory_path=directory_path), \
                                                                                                             config=configIN)
    all_gamma_holes.append(vars()["gamma_holes_"+str(number_str)])
    all_photoemission_electrons.append(vars()["photoemission_electrons_inside_"+str(number_str)])
    all_protons_inside.append(vars()["protons_inside_"+str(number_str)])
    all_electrons_inside.append(vars()["electrons_inside_"+str(number_str)])
    print(78*"-")

# Concatenate all iterations into single DataFrames
all_gamma_holes_df = pd.concat(all_gamma_holes, ignore_index=True)
all_photoemission_electrons_df = pd.concat(all_photoemission_electrons, ignore_index=True)
all_protons_inside_df = pd.concat(all_protons_inside, ignore_index=True)
all_electrons_inside_df = pd.concat(all_electrons_inside, ignore_index=True)

In [ ]:
# Concatenate all iterations into single DataFrames
all_gamma_holes_df = pd.concat(all_gamma_holes, ignore_index=True)
all_photoemission_electrons_df = pd.concat(all_photoemission_electrons, ignore_index=True)
all_protons_inside_df = pd.concat(all_protons_inside, ignore_index=True)
all_electrons_inside_df = pd.concat(all_electrons_inside, ignore_index=True)

In [ ]:
print("up through iteration 28")
# input order: gammas, photoelectrons, protons, electrons, convex_combined,
surface,facecolors = plot_surface_potential_allparticle_case(all_gamma_holes_df, all_photoemission_electrons_df, all_protons_inside_df, all_electrons_inside_df, 
                                                      stacked_spheres, vmin=-0.2,vmax=0.2)
print(min(facecolors),max(facecolors))
plt.plot(facecolors[facecolors!=0])
plt.show()

# Make sure each triangle has its own unique vertices
surface_edited = surface.copy()
surface_edited.unmerge_vertices()
surface_edited.visual.vertex_colors = None
surface_edited.show()

In [ ]:
bbox_min = np.array([-0.25, -0.31, 0])
bbox_max = np.array([ 0.15,  0.1,  100])

# Filter vertices
in_box = np.all((surface_edited.vertices >= bbox_min) & 
                (surface_edited.vertices <= bbox_max), axis=1)

# Get face indices where all 3 vertices are in the box
face_mask = np.all(in_box[surface_edited.faces], axis=1)

# Extract submesh
cropped = surface_edited.submesh([face_mask], only_watertight=False, append=True)
cropped.show()

In [ ]:
# Define colormap and normalization
cmap = plt.cm.seismic
norm = Normalize(vmin=-0.2, vmax=0.2)

# Create a figure and a single axis for the colorbar
fig, ax = plt.subplots(figsize=(4, 0.5))
fig.subplots_adjust(bottom=0.5)

# Create the colorbar
cb = ColorbarBase(ax, cmap=cmap, norm=norm, orientation='horizontal')
cb.set_label('Surface Potential (mV)')  # Optional label

plt.show()


In [ ]:
print("up through iteration 19: only SW ions")
surface,facecolors = plot_surface_potential_fornegativepositive_charge(all_electrons_inside_df, all_protons_inside_df, stacked_spheres, vmin=-0.2,vmax=0.2)

print(min(facecolors),max(facecolors))
plt.plot(facecolors[facecolors!=0])
plt.show()

# Make sure each triangle has its own unique vertices
surface_edited = surface.copy()
surface_edited.unmerge_vertices()
surface_edited.visual.vertex_colors = None
surface_edited.show()

In [ ]:
print("up through iteration 22: only photons")
surface,facecolors = plot_surface_potential_fornegativepositive_charge(all_photoemission_electrons_df, all_gamma_holes_df, stacked_spheres, vmin=-0.5,vmax=0.5)

print(min(facecolors),max(facecolors))
plt.plot(facecolors[facecolors!=0])
plt.show()

# Make sure each triangle has its own unique vertices
surface_edited = surface.copy()
surface_edited.unmerge_vertices()
surface_edited.visual.vertex_colors = None
surface_edited.show()

In [ ]:
bbox_min = np.array([-0.1, -0.1, 0])
bbox_max = np.array([ 0.15,  0.15,  200])

# Filter vertices
in_box = np.all((surface_edited.vertices >= bbox_min) & 
                (surface_edited.vertices <= bbox_max), axis=1)

# Get face indices where all 3 vertices are in the box
face_mask = np.all(in_box[surface_edited.faces], axis=1)

# Extract submesh
cropped = surface_edited.submesh([face_mask], only_watertight=False, append=True)
cropped.show()
